11.11.2022

# Test Grad-CAM on UNet model

Provo a usare una Grad-CAM su un modello salvato della UNet e il movie 34 (dove la fine della wave viene detettata come puff).

In [1]:
import configparser
import logging
import os
import sys

import numpy as np
import torch
from architectures import TempRedUNet
from datasets import SparkDataset
from in_out_tools import write_videos_on_disk
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from training_inference_tools import run_samples_in_model

import unet

### Set parameters

In [2]:
training_name = 'TEMP_new_annotated_peaks_physio'
config_file = 'config_temp_new_annotated_peaks_physio.ini'

print(f"Processing training '{training_name}'...")

Processing training 'TEMP_new_annotated_peaks_physio'...


### Configure output folder

In [3]:
output_folder = "trainings_validation" # same folder for train and test preds
os.makedirs(output_folder, exist_ok=True)

# subdirectory of output_folder where predictions are saved
# change this to save results for same model with different inference approaches
output_name = training_name

save_folder = os.path.join(output_folder, output_name, "gradCAM")
os.makedirs(save_folder, exist_ok=True)

print(f"Output files will be saved on '{save_folder}'")

Output files will be saved on 'trainings_validation\TEMP_new_annotated_peaks_physio\gradCAM'


### Detect GPU, if available

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpus = torch.cuda.device_count()
print(f"Using device '{device}' with {n_gpus} GPUs")

Using device 'cuda' with 1 GPUs


### Load config file

In [5]:
config_folder = "config_files"
CONFIG_FILE = os.path.join(config_folder, config_file)
c = configparser.ConfigParser()

print(f"Loading {CONFIG_FILE}")
c.read(CONFIG_FILE)

Loading config_files\config_temp_new_annotated_peaks_physio.ini


['config_files\\config_temp_new_annotated_peaks_physio.ini']

### Config dataset and UNet model

In [6]:
### Params ###
load_epoch = c.getint("testing", "load_epoch")

batch_size = c.getint("testing", "batch_size", fallback="1")
ignore_frames = c.getint("training", "ignore_frames_loss")

temporal_reduction = c.getboolean("network", "temporal_reduction", fallback=False)
num_channels = c.getint("network", "num_channels", fallback=1) if temporal_reduction else 1

In [7]:
### Configure sample input ### 

sample_ids = ["34"]

dataset_path = os.path.realpath(c.get("dataset", "relative_path"))
assert os.path.isdir(dataset_path), f"\"{dataset_path}\" is not a directory"
print(f"Using {dataset_path} as dataset root path")


Using C:\Users\prisc\Code\sparks_project\data\sparks_dataset as dataset root path


In [8]:
### Configure inference method and parameters ###

data_step = c.getint("testing", "data_step")
data_duration = c.getint("testing", "data_duration")
inference = c.get("testing", "inference")

In [9]:
### Configure UNet ###

batch_norm = {'batch': True, 'none': False}

unet_config = unet.UNetConfig(
    steps=c.getint("network", "unet_steps"),
    first_layer_channels=c.getint("network", "first_layer_channels"),
    num_classes=4,
    ndims=3,
    dilation=c.getint("network", "dilation", fallback=1),
    border_mode=c.get("network", "border_mode"),
    batch_normalization=batch_norm[c.get("network", "batch_normalization")],
    num_input_channels=num_channels,
)
if not temporal_reduction:
    network = unet.UNetClassifier(unet_config)
else:
    assert c.getint("dataset", "data_duration") % num_channels == 0, \
    "using temporal reduction chunks_duration must be a multiple of num_channels"
    network = TempRedUNet(unet_config)

network = nn.DataParallel(network).to(device)

In [13]:
### Load UNet model ###
models_relative_path = 'runs/'
model_path = os.path.join(models_relative_path, training_name)
#logger.info(f"Saved model path: {model_path}")
summary_writer = SummaryWriter(os.path.join(model_path, "summary"),
                               purge_step=0)

trainer = unet.TrainingManager(
        # training items
        training_step = None,
        save_path=model_path,
        managed_objects=unet.managed_objects({'network': network}),
        summary_writer=summary_writer
    )

print(f"Loading trained model '{training_name}' at epoch {load_epoch}...")
trainer.load(load_epoch)

Loading trained model 'TEMP_new_annotated_peaks_physio' at epoch 100000...


### Configure Grad-CAM

In [19]:
from torchsummary import summary

summary(network, (1,32,64,512))

In [ ]:
network = medcam.inject(network,
                        output_dir='attention_maps',
                        backend='gcam',
                        layer='layer4',
                        label='best',
                        save_maps=True)